In [3]:
import os
cache_dir = "/scratches/dialfs/alta/hln35/.cache"
os.environ['TRANSFORMERS_CACHE'] = '/scratches/dialfs/alta/hln35/.cache'

In [11]:
PATH1 = 'model/flant5_small_lr_10-5_wd_10-2_qa/checkpoint-3378/'
PATH2 = 'model/flant5_small_lr_10-5_wd_10-2_qa/checkpoint-2252/'
PATH3 = 'model/flant5_small_lr_10-5_wd_10-2_qa/checkpoint-1126/'



In [36]:
model_small = "google/flan-t5-small"

In [5]:
from datasets import load_dataset

data_points = load_dataset("ai2_arc", "ARC-Easy", split='test', cache_dir=cache_dir)

In [6]:
data_points

Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 2376
})

In [7]:
data_points[0]

{'id': 'Mercury_417466',
 'question': 'Which statement best explains why photosynthesis is the foundation of most food webs?',
 'choices': {'text': ['Sunlight is the source of energy for nearly all ecosystems.',
   'Most ecosystems are found on land instead of in water.',
   'Carbon dioxide is more available than other gases.',
   'The producers in all ecosystems are plants.'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'A'}

In [16]:
prefix = "context: for each questions there are a few choices of answer. answer this question by choosing the best choice: "


def preprocess_function(data_points):
    # inputs = [prefix + example[source_lang] for example in examples["translation"]]
    # targets = [example[target_lang] for example in examples["translation"]]
    # model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    # model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    inputs = []
    for i in range(len(data_points["question"])):
        q = data_points["question"][i]
        choices = ""
        choice = ""
        for t in range(len(data_points["choices"][i]["label"])):
            choices += data_points["choices"][i]["label"][t] + " " + data_points["choices"][i]["text"][t] + ". "
            
        text = prefix + q + "Choices: " + choices
        inputs.append(text)
    model_inputs = tokenizer(inputs, truncation=True)
    
    return model_inputs

In [9]:
import evaluate

metric = evaluate.load("accuracy", cache_dir=cache_dir)

In [40]:
from transformers import AutoTokenizer
import torch
from transformers import AutoModelForQuestionAnswering, AutoModel, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(PATH1, local_files_only=True)
model1 = AutoModelForSeq2SeqLM.from_pretrained(PATH1, local_files_only=True)
model2 = AutoModelForSeq2SeqLM.from_pretrained(PATH2, local_files_only=True)
model3 = AutoModelForSeq2SeqLM.from_pretrained(PATH3, local_files_only=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_small)


In [20]:
import json, re
import numpy as np

In [14]:
ans_id_dict = {71: "A", 272: "B", 205: "C", 309: "D"}
index_to_ans = {0: "A", 1: "B", 2: "C", 3: "D"}

In [17]:
tokenized_datasets = data_points.map(preprocess_function, batched=True)

Map:   0%|          | 0/2376 [00:00<?, ? examples/s]

In [41]:
input_ids = tokenized_datasets["input_ids"]
labels = tokenized_datasets['answerKey']
model_outputs = []
probability_output = []
results = {}
group_len = 20
for i in range(0, len(input_ids)):
        test_tensor = torch.tensor([input_ids[i]])
        preds = model1(input_ids=test_tensor, decoder_input_ids=torch.tensor([[model1.config.decoder_start_token_id,]]))      
        preds_prob = []
        for t in ans_id_dict.keys():
            preds_prob.append(torch.nn.functional.softmax(preds.logits, dim=-1)[...,t][0][0].item())
            
        model_outputs.append(index_to_ans[np.argmax(preds_prob)])
        probability_output.append(preds_prob)
# result = metric.compute(predictions=model_outputs, references=labels)


In [44]:
input_ids[0]

[2625,
 10,
 21,
 284,
 746,
 132,
 33,
 3,
 9,
 360,
 3703,
 13,
 1525,
 5,
 1525,
 48,
 822,
 57,
 4622,
 8,
 200,
 1160,
 10,
 4073,
 2493,
 200,
 3,
 9453,
 572,
 1202,
 17282,
 19,
 8,
 3361,
 13,
 167,
 542,
 765,
 7,
 58,
 3541,
 32,
 867,
 7,
 10,
 71,
 3068,
 2242,
 19,
 8,
 1391,
 13,
 827,
 21,
 2111,
 66,
 10789,
 7,
 5,
 5,
 272,
 1377,
 10789,
 7,
 33,
 435,
 30,
 1322,
 1446,
 13,
 16,
 387,
 5,
 5,
 205,
 14335,
 22122,
 19,
 72,
 347,
 145,
 119,
 24436,
 5,
 5,
 309,
 37,
 10850,
 16,
 66,
 10789,
 7,
 33,
 2677,
 5,
 5,
 3,
 1]

In [42]:
model_outputs

['A',
 'A',
 'D',
 'D',
 'A',
 'D',
 'B',
 'A',
 'A',
 'A',
 'B',
 'A',
 'D',
 'B',
 'A',
 'D',
 'A',
 'C',
 'A',
 'C',
 'A',
 'B',
 'A',
 'A',
 'C',
 'A',
 'A',
 'D',
 'A',
 'B',
 'A',
 'C',
 'D',
 'A',
 'A',
 'D',
 'D',
 'C',
 'C',
 'B',
 'A',
 'C',
 'C',
 'C',
 'A',
 'D',
 'A',
 'D',
 'C',
 'D',
 'A',
 'D',
 'C',
 'A',
 'C',
 'D',
 'B',
 'D',
 'A',
 'D',
 'A',
 'D',
 'A',
 'A',
 'A',
 'B',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'D',
 'C',
 'C',
 'D',
 'C',
 'B',
 'A',
 'B',
 'C',
 'D',
 'C',
 'D',
 'B',
 'A',
 'D',
 'C',
 'C',
 'A',
 'A',
 'C',
 'A',
 'A',
 'B',
 'B',
 'D',
 'D',
 'D',
 'A',
 'C',
 'D',
 'D',
 'A',
 'A',
 'B',
 'A',
 'B',
 'C',
 'A',
 'A',
 'A',
 'B',
 'D',
 'C',
 'B',
 'C',
 'A',
 'A',
 'D',
 'D',
 'D',
 'B',
 'D',
 'A',
 'C',
 'A',
 'A',
 'A',
 'C',
 'C',
 'A',
 'C',
 'C',
 'A',
 'A',
 'A',
 'C',
 'D',
 'B',
 'A',
 'C',
 'C',
 'C',
 'A',
 'C',
 'A',
 'C',
 'C',
 'D',
 'C',
 'C',
 'A',
 'A',
 'A',
 'C',
 'B',
 'D',
 'C',
 'B',
 'B',
 'D',
 'C',
 'C',
 'A',
 'D',
 'D'

In [24]:
labels

['A',
 'B',
 'D',
 'D',
 'B',
 'C',
 'A',
 'C',
 'C',
 'A',
 'B',
 'B',
 'B',
 'B',
 '2',
 'B',
 'B',
 'D',
 'B',
 'D',
 'D',
 'D',
 'D',
 'C',
 'B',
 'C',
 '2',
 'B',
 'A',
 'B',
 'A',
 'C',
 'A',
 'A',
 'C',
 'A',
 'D',
 'C',
 'C',
 'B',
 'A',
 'B',
 'A',
 'C',
 'A',
 'D',
 'D',
 'C',
 'D',
 'D',
 'A',
 'B',
 'C',
 'A',
 'C',
 'B',
 'D',
 'C',
 'B',
 'A',
 'B',
 'D',
 '4',
 '2',
 'A',
 'A',
 'B',
 'B',
 'A',
 'B',
 'A',
 'D',
 'D',
 'B',
 'C',
 'D',
 'B',
 'B',
 'B',
 'A',
 'D',
 'A',
 'C',
 'A',
 'B',
 'A',
 'A',
 'A',
 'C',
 'D',
 'C',
 '3',
 '3',
 'B',
 'B',
 'A',
 'A',
 'C',
 'B',
 'C',
 'B',
 'D',
 'B',
 'C',
 'D',
 'C',
 'B',
 'C',
 'C',
 'C',
 'B',
 'B',
 'B',
 'C',
 'C',
 'B',
 'B',
 'B',
 'A',
 'A',
 'A',
 'C',
 'B',
 'C',
 'C',
 'C',
 'A',
 'C',
 'C',
 'B',
 'C',
 'A',
 'B',
 'A',
 'B',
 'D',
 'A',
 'B',
 'D',
 'C',
 'B',
 'D',
 'B',
 'D',
 'A',
 'D',
 'A',
 'B',
 'B',
 'A',
 'A',
 '1',
 'D',
 'A',
 'D',
 'D',
 'D',
 'A',
 'D',
 'B',
 'D',
 'B',
 'C',
 'D',
 'C',
 'A',
 'B'

In [43]:
probability_output

[[0.027477705851197243,
  0.004493290092796087,
  0.013242924585938454,
  0.005709340795874596],
 [0.569154679775238,
  0.24304603040218353,
  0.13723702728748322,
  0.038229797035455704],
 [0.01982533000409603,
  0.04822323098778725,
  0.23748210072517395,
  0.6364709734916687],
 [0.06563570350408554,
  0.11201649159193039,
  0.2747131288051605,
  0.5397303104400635],
 [0.3756766617298126,
  0.1064329519867897,
  0.3187275826931,
  0.10417696088552475],
 [0.11620365083217621,
  0.0802730917930603,
  0.06037762016057968,
  0.7404991388320923],
 [0.2519770860671997,
  0.3338797986507416,
  0.2738460898399353,
  0.04245809465646744],
 [0.363213449716568,
  0.20839367806911469,
  0.2654888331890106,
  0.15753714740276337],
 [0.41071656346321106,
  0.10037323087453842,
  0.18329383432865143,
  0.2931010127067566],
 [0.4037216603755951,
  0.14883820712566376,
  0.0978614017367363,
  0.2789672315120697],
 [0.2649528682231903,
  0.26577654480934143,
  0.21917852759361267,
  0.1463217139244079

In [26]:
result1 = 0
for i in range(min(len(model_outputs), len(labels))):
    if model_outputs[i] == labels[i]:
        result1 += 1
result1

772

In [33]:
model_outputs = []
probability_output2 = []
for i in range(0, len(input_ids)):
        test_tensor = torch.tensor([input_ids[i]])
        preds = model2(input_ids=test_tensor, decoder_input_ids=torch.tensor([[model1.config.decoder_start_token_id,]]))      
        preds_prob = []
        for t in ans_id_dict.keys():
            preds_prob.append(torch.nn.functional.softmax(preds.logits, dim=-1)[...,t][0][0].item())
            
        model_outputs.append(index_to_ans[np.argmax(preds_prob)])
        probability_output2.append(preds_prob)
        
result2 = 0
for i in range(min(len(model_outputs), len(labels))):
    if model_outputs[i] == labels[i]:
        result2 += 1
result2

772

In [34]:
probability_output2

[[0.027477705851197243,
  0.004493290092796087,
  0.013242924585938454,
  0.005709340795874596],
 [0.569154679775238,
  0.24304603040218353,
  0.13723702728748322,
  0.038229797035455704],
 [0.01982533000409603,
  0.04822323098778725,
  0.23748210072517395,
  0.6364709734916687],
 [0.06563570350408554,
  0.11201649159193039,
  0.2747131288051605,
  0.5397303104400635],
 [0.3756766617298126,
  0.1064329519867897,
  0.3187275826931,
  0.10417696088552475],
 [0.11620365083217621,
  0.0802730917930603,
  0.06037762016057968,
  0.7404991388320923],
 [0.2519770860671997,
  0.3338797986507416,
  0.2738460898399353,
  0.04245809465646744],
 [0.363213449716568,
  0.20839367806911469,
  0.2654888331890106,
  0.15753714740276337],
 [0.41071656346321106,
  0.10037323087453842,
  0.18329383432865143,
  0.2931010127067566],
 [0.4037216603755951,
  0.14883820712566376,
  0.0978614017367363,
  0.2789672315120697],
 [0.2649528682231903,
  0.26577654480934143,
  0.21917852759361267,
  0.1463217139244079

In [35]:
probability_output

[[0.027477705851197243,
  0.004493290092796087,
  0.013242924585938454,
  0.005709340795874596],
 [0.569154679775238,
  0.24304603040218353,
  0.13723702728748322,
  0.038229797035455704],
 [0.01982533000409603,
  0.04822323098778725,
  0.23748210072517395,
  0.6364709734916687],
 [0.06563570350408554,
  0.11201649159193039,
  0.2747131288051605,
  0.5397303104400635],
 [0.3756766617298126,
  0.1064329519867897,
  0.3187275826931,
  0.10417696088552475],
 [0.11620365083217621,
  0.0802730917930603,
  0.06037762016057968,
  0.7404991388320923],
 [0.2519770860671997,
  0.3338797986507416,
  0.2738460898399353,
  0.04245809465646744],
 [0.363213449716568,
  0.20839367806911469,
  0.2654888331890106,
  0.15753714740276337],
 [0.41071656346321106,
  0.10037323087453842,
  0.18329383432865143,
  0.2931010127067566],
 [0.4037216603755951,
  0.14883820712566376,
  0.0978614017367363,
  0.2789672315120697],
 [0.2649528682231903,
  0.26577654480934143,
  0.21917852759361267,
  0.1463217139244079

In [28]:
model_outputs = []
for i in range(0, len(input_ids)):
        test_tensor = torch.tensor([input_ids[i]])
        preds = model3(input_ids=test_tensor, decoder_input_ids=torch.tensor([[model1.config.decoder_start_token_id,]]))      
        preds_prob = []
        for t in ans_id_dict.keys():
            preds_prob.append(torch.nn.functional.softmax(preds.logits, dim=-1)[...,t][0][0].item())
            
        model_outputs.append(index_to_ans[np.argmax(preds_prob)])
result3 = 0
for i in range(min(len(model_outputs), len(labels))):
    if model_outputs[i] == labels[i]:
        result3 += 1
result3

772

In [38]:
model_outputs = []
probability_output = []
for i in range(0, len(input_ids)):
        test_tensor = torch.tensor([input_ids[i]])
        preds = model(input_ids=test_tensor, decoder_input_ids=torch.tensor([[model1.config.decoder_start_token_id,]]))      
        preds_prob = []
        for t in ans_id_dict.keys():
            preds_prob.append(torch.nn.functional.softmax(preds.logits, dim=-1)[...,t][0][0].item())
            
        model_outputs.append(index_to_ans[np.argmax(preds_prob)])
        probability_output.append(preds_prob)
        
result = 0
for i in range(min(len(model_outputs), len(labels))):
    if model_outputs[i] == labels[i]:
        result += 1
result

772

In [39]:
probability_output

[[0.027477705851197243,
  0.004493290092796087,
  0.013242924585938454,
  0.005709340795874596],
 [0.569154679775238,
  0.24304603040218353,
  0.13723702728748322,
  0.038229797035455704],
 [0.01982533000409603,
  0.04822323098778725,
  0.23748210072517395,
  0.6364709734916687],
 [0.06563570350408554,
  0.11201649159193039,
  0.2747131288051605,
  0.5397303104400635],
 [0.3756766617298126,
  0.1064329519867897,
  0.3187275826931,
  0.10417696088552475],
 [0.11620365083217621,
  0.0802730917930603,
  0.06037762016057968,
  0.7404991388320923],
 [0.2519770860671997,
  0.3338797986507416,
  0.2738460898399353,
  0.04245809465646744],
 [0.363213449716568,
  0.20839367806911469,
  0.2654888331890106,
  0.15753714740276337],
 [0.41071656346321106,
  0.10037323087453842,
  0.18329383432865143,
  0.2931010127067566],
 [0.4037216603755951,
  0.14883820712566376,
  0.0978614017367363,
  0.2789672315120697],
 [0.2649528682231903,
  0.26577654480934143,
  0.21917852759361267,
  0.1463217139244079

In [37]:
print(f"Total points are {len(data_points)}. Number of correct answers is {correct}. Number of empty answers is {empty}")

Total points are 2376. Number of correct answers is 722. Number of empty answers is 162


In [39]:
model_large = "google/flan-t5-large"
tokenizer_large = AutoTokenizer.from_pretrained(model_large)
model_large = AutoModelForSeq2SeqLM.from_pretrained(model_large)

In [40]:
correct = 0
empty = 0
pattern = r'<pad>\s+(.*?)</s>'  

for i in range(len(data_points)):
    question = data_points[i]["question"]
    choices = ""
    choice = ""
    preds_strip = ""
    for t in range(len(data_points[i]["choices"]["label"])):
        # choices += data_points[i]["choices"]["label"][t] + " " + data_points[i]["choices"]["text"][t] + ". "
        choices += data_points[i]["choices"]["text"][t] + ". "
        
    text = prefix + question + "Choices: " + choices
    ref = data_points[i]["answerKey"]
    inputs = tokenizer_large(text, return_tensors="pt").input_ids
    preds_tokenized = model_large.generate(inputs, max_new_tokens=128, do_sample=False) 
    preds = tokenizer_large.batch_decode(preds_tokenized)

    match = re.search(pattern, preds[0]) 

    if match:
        preds_strip = match.group(1)
    
    for t in range(len(data_points[i]["choices"]["label"])):
        if preds_strip in data_points[i]["choices"]["text"][t]:
            choice = data_points[i]["choices"]["label"][t]
    if choice == ref:
        correct += 1
    elif choice == "":
        empty += 1

In [41]:
print(f"Total points are {len(data_points)}. Number of correct answers is {correct}. Number of empty answers is {empty}")

Total points are 2376. Number of correct answers is 1675. Number of empty answers is 105
